## **State Space** 

Please stay tuned, here we will have the state space representations of the models...

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend
from darli.robots import biped
from darli.functional import Functional

# from darli.model import Model

# from robot_descriptions import z1_description

# model = Functional(CasadiBackend(z1_description.URDF_PATH))
# model = Robot(CasadiBackend(z1_description.URDF_PATH))

## **Prespecified Robots**

One can build different robots....

As example let us consider the Atlas humanoid robot:

Note: robot loaded in example is `fixed` in its pelvis and in real world, you have to create a floating base model to have a full set of generalized coordinates.

In [3]:
from robot_descriptions import atlas_v4_description

biped_urdf = atlas_v4_description.URDF_PATH

biped_model = biped(
    Functional,
    CasadiBackend,
    biped_urdf,
    torso={"torso": "pelvis"},
    foots={
        "left_foot": "l_foot",
        "right_foot": "r_foot",
    },
)

In [4]:
import numpy as np

In [5]:
biped_model.nu

24

In [6]:
biped_model.nq

30

In [7]:
biped_model.body("left_foot").contact.qforce

Function(qforce:(i0[30],i1[6])->(o0[30]) SXFunction)

In [8]:
biped_model.contact_qforce

Function(contact_qforce:(q[30],l_foot[6],r_foot[6])->(contact_qforce[30]) SXFunction)

In [9]:
biped_model.inverse_dynamics

Function(inverse_dynamics:(q[30],v[30],dv[30],l_foot[6],r_foot[6])->(tau[30]) SXFunction)

In [10]:
biped_model.body("torso").position

Function(position:(i0[30])->(o0[3]) SXFunction)

In [11]:
biped_model.body("torso").rotation

Function(rotation:(i0[30])->(o0[3x3]) SXFunction)

In [12]:
biped_model.body("torso").jacobian.world_aligned

Function(jacobian_world_aligned:(i0[30])->(o0[6x30]) SXFunction)

In [13]:
biped_model.body("torso").angular_velocity.local

Function(angular_velocity_local:(i0[30],i1[30])->(o0[3]) SXFunction)

In [14]:
biped_model.body("torso").jacobian_dt.world_aligned

Function(jacobian_dt_world_aligned:(i0[30],i1[30])->(o0[6x30]) SXFunction)

In [15]:
biped_model.body("left_foot").contact.jacobian

Function(jacobian:(i0[30])->(o0[30x6]) SXFunction)

In [16]:
biped_model.body("right_foot").contact.jacobian

Function(jacobian:(i0[30])->(o0[30x6]) SXFunction)

In [17]:
q_next = biped_model.backend.integrate_configuration(dt=0.001)

In [18]:
import casadi as cs

q_next = biped_model.backend.integrate_configuration(dt=cs.SX.sym("dt", 1))

#### **State space Representation and Linearization**

One can easily transform the mechanical system to the state space form by defining the state $\mathbf{x} = [\mathbf{q}, \mathbf{v}]^T$:


$$
\dot{\mathbf{x}}= \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\dot{\mathbf{v}}
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{q})\mathbf{v} \\ 
\text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{S}\mathbf{u})
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{x}_1)\mathbf{x}_2 \\
\text{aba}(\mathbf{x}_1, \mathbf{x}_2, \mathbf{S}\mathbf{u})
\end{bmatrix}
$$



In [ ]:
model.state_space

The above equation can be easily linearized to produce following linear approximation:

One can easily find linearization with respect to state:

In [ ]:
model.state_space.state_derivative

Function(state_derivative:(q[6],v[6],tau[3])->(state_derivative[12]) SXFunction)

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3])->(state_jacobian[12x12,72nz]) SXFunction)

and control:

In [ ]:
model.state_space.input_jacobian

Function(input_jacobian:(q[6],v[6],tau[3])->(input_jacobian[12x3,18nz]) SXFunction)

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06")

One may also retrieve a hash map of all bodies:

In [ ]:
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06").linear_acceleration

FrameQuantity(local=Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world=Function(linear_acceleration_world:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world_aligned=Function(linear_acceleration_world_aligned:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction))

In [ ]:
model.body("link06").linear_acceleration.local

Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

In [ ]:
model.body("link06").jacobian.local

Function(jacobian_local:(i0[6])->(o0[6x6]) SXFunction)

In [ ]:
model.body("link06").jacobian.local
model.body("link06").jacobian_dt.local
model.body("link06").linear_velocity.local
model.body("link06").angular_velocity.local
model.body("link06").linear_acceleration.local
model.body("link06").angular_acceleration.local

Function(angular_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Function(jacobian_world:(i0[6])->(o0[6x6]) SXFunction)

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Function(qforce:(i0[6],i1[6])->(o0[6]) SXFunction)

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3],link06[6])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

Function(state_derivative:(q[6],v[6],tau[3],link06[6])->(state_derivative[12]) SXFunction)

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3],link06[6])->(state_jacobian[12x12,78nz]) SXFunction)

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

Function(nonlin_wrench_cone:(force[6])->(constraint[6]) SXFunction)

In [ ]:
model.body("link06").contact.cone.linear()

DM(
[[-1, 0, -0.5, 0, 0, 0], 
 [1, 0, -0.5, 0, 0, 0], 
 [0, -1, -0.5, 0, 0, 0], 
 [0, 1, -0.5, 0, 0, 0], 
 [0, 0, -0.02, -1, 0, 0], 
 [0, 0, -0.02, 1, 0, 0], 
 [0, 0, -0.05, 0, -1, 0], 
 [0, 0, -0.05, 0, 1, 0], 
 [-0.02, -0.05, -0.035, 0.5, 0.5, -1], 
 [-0.02, 0.05, -0.035, 0.5, -0.5, -1], 
 [0.02, -0.05, -0.035, -0.5, 0.5, -1], 
 [0.02, 0.05, -0.035, -0.5, -0.5, -1], 
 [0.02, 0.05, -0.035, 0.5, 0.5, 1], 
 [0.02, -0.05, -0.035, 0.5, -0.5, 1], 
 [-0.02, 0.05, -0.035, -0.5, 0.5, 1], 
 [-0.02, -0.05, -0.035, -0.5, -0.5, 1]])

In [ ]:
model.add_body({"ee": "link06"})
model.bodies.keys()

dict_keys(['link06', 'ee'])

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.